In [52]:
import h2o
from h2o.estimators.xgboost import H2OXGBoostEstimator

In [53]:
h2o.connect(ip="104.196.36.245")

Connecting to H2O server at http://104.196.36.245:54321... successful.


H2O cluster uptime:,16 mins 39 secs
H2O cluster version:,3.13.0.313
H2O cluster version age:,2 days
H2O cluster name:,olmsteadbrett
H2O cluster total nodes:,1
H2O cluster free memory:,1.614 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://104.196.36.245:54321
H2O connection proxy:,None


<H2OConnection to http://104.196.36.245:54321, session _sid_bab2>

In [54]:
# Import some data from Amazon S3
h2oDF = h2o.import_file("https://s3-us-west-1.amazonaws.com/dsclouddata/LendingClubData/LoansGoodBad.csv")

# Stratified Split into Train/Test
stratsplit = h2oDF["Bad_Loan"].stratified_split(test_frac=0.3, seed=12349453)
train = h2oDF[stratsplit=="train"]
test = h2oDF[stratsplit=="test"]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [55]:
# Identify predictors and response
x = train.columns
y = "Bad_Loan"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [56]:
train.head(5)

RowID,Loan_Amount,Term,Interest_Rate,Employment_Years,Home_Ownership,Annual_Income,Verification_Status,Loan_Purpose,State,Debt_to_Income,Delinquent_2yr,Revolving_Cr_Util,Total_Accounts,Bad_Loan,Longest_Credit_Length
2,2500,60 months,15.27,0.5,RENT,30000,VERIFIED - income source,car,GA,1,0,9.4,4,BAD,12
3,2400,36 months,15.96,10,RENT,12252,not verified,small_business,IL,8.72,0,98.5,10,GOOD,10
4,10000,36 months,13.49,10,RENT,49200,VERIFIED - income source,other,CA,20,0,21,37,GOOD,15
5,5000,36 months,7.9,3,RENT,36000,VERIFIED - income source,wedding,AZ,11.2,0,28.3,12,GOOD,7
6,3000,36 months,18.64,9,RENT,48000,VERIFIED - income source,car,CA,5.35,0,87.5,4,GOOD,4


In [62]:
%%time
XGB_GPU = H2OXGBoostEstimator(model_id="XGB_on_GPU", ntrees=500, max_depth=9, learn_rate=0.05, backend="gpu", gpu_id=0)
XGB_GPU.train(x=x, y=y, training_frame=train, validation_frame=test)
XGB.auc()

xgboost Model Build progress: |███████████████████████████████████████████| 100%
CPU times: user 224 ms, sys: 36 ms, total: 260 ms
Wall time: 20.8 s


In [63]:
XGB_GPU.auc()

0.8965164265157294

In [60]:
%%time
XGB_CPU = H2OXGBoostEstimator(model_id="XGB_on_CPU", ntrees=500, max_depth=9, learn_rate=0.05, backend="cpu")
XGB_CPU.train(x=x, y=y, training_frame=train, validation_frame=test)

xgboost Model Build progress: |███████████████████████████████████████████| 100%
CPU times: user 644 ms, sys: 68 ms, total: 712 ms
Wall time: 3min 27s


In [65]:
XGB_CPU.auc()

0.8799878126885237